# Import mongo db pakg

In [ ]:
import os
import json
import glob
import uuid
import datetime
from pymongo import MongoClient
import pandas as pd

# Class to build database

In [ ]:
class DataManager:
    def __init__(self, connection_string='mongodb://localhost:27017/', db_name='kaggle_platform'):
        """Initialize connection to MongoDB."""
        self.client = MongoClient(connection_string)
        self.db = self.client[db_name]
        
        # Initialize collections
        self.competitions = self.db['competitions']
        self.notebooks = self.db['notebooks']
        self.datasets = self.db['datasets']
        self.user_profiles = self.db['user_profiles']
        self.competition_history = self.db['competition_history']
        
        # Create indexes for faster lookups
        self.competitions.create_index('competition_id', unique=True)
        self.notebooks.create_index([('notebook_id', 1), ('competition_id', 1)], unique=True)
        self.datasets.create_index([('dataset_id', 1), ('competition_id', 1)], unique=True)
        self.user_profiles.create_index('user_id', unique=True)
        self.competition_history.create_index([('user_id', 1)], unique=True)  # One active competition per user

    # Competition methods
    def create_competition(self, record):
        """Create a new competition entry."""
        try:
            competition = {
                'competition_id': record.get('competition_id'),
                'title': record.get('title'),
                'description': record.get('description'),
                'evaluation': datetime.datetime.now(),
                'competition_host': record.get('competition_host', datetime.datetime.now()),
                'price_award': record.get('price_award'),
                "entrants": record.get("entrants"),
                "participants": record.get("participants"),
                "teams": record.get("teams"),
                "submissions": record.get("submissions"),
                "tags": record.get("tags"),
                "competition_url": record.get("competition_url"),
                "data_description": record.get("data_description"),
                "data_files_num": record.get("data_files_num"),
                "data_size": record.get("data_size"),
                "data_type": record.get("data_type"),
            }
            
            self.competitions.update_one(
                {'competition_id': competition['competition_id']},
                {'$set': competition},
                upsert=True
            )
            
            return True
        
        except Exception as e:
            print(f"Error creating competition: {e}")
            return False

    # Notebook methods
    def import_notebook(self, file_path, competition_id,metadata=None):
        """Import a single Jupyter notebook (.ipynb) file into MongoDB and associate with a competition."""
        try:
            # Check if competition exists
            if not self.get_competition(competition_id):
                print(f"Competition {competition_id} doesn't exist")
                return False
                
            # Check if file exists and is a Jupyter notebook
            if not os.path.isfile(file_path):
                print(f"File {file_path} does not exist")
                return False
            
            # Extract filename without extension as notebook_id
            notebook_id = os.path.splitext(os.path.basename(file_path))[0]
            
            # Read the notebook file
            with open(file_path, 'r', encoding='utf-8') as f:
                notebook_content = json.load(f)
            # Prepare notebook document with metadata
            notebook_doc = {
                'notebook_id': notebook_id,
                'competition_id': competition_id,
                'content': notebook_content,
                'last_updated': datetime.datetime.now()
            }
            
            # Add metadata if available
            if metadata:
                notebook_doc['metadata'] = {
                    'url': metadata.get('url', ''),
                    'score': metadata.get('score', 0),
                    'votes': metadata.get('votes', 0),
                    'comments': metadata.get('comments', 0),
                    'date_created': metadata.get('date_created', ''),
                    'sort_by': metadata.get('sort_by', '')
                }
            
            # Store the notebook in MongoDB with competition association
            self.notebooks.update_one(
                {'notebook_id': notebook_id, 'competition_id': competition_id},
                {'$set': notebook_doc},
                upsert=True
            )
            return True
            
        except Exception as e:
            print(f"Error importing notebook {file_path}: {e}")
            return False
    
    def import_notebook_folder(self, folder_path, competition_id):
        """Import all Jupyter notebooks from a folder into MongoDB for a specific competition."""
        # Make sure folder path exists
        if not os.path.isdir(folder_path):
            print(f"Error: {folder_path} is not a valid directory")
            return 0, 0
        metadata_file = os.path.join(folder_path,competition_id, "metadata", "all_notebooks_metadata.json")
        metadata_dict = {}
        
        # Load metadata if available
        with open(metadata_file, 'r') as f:
            metadata_dict = json.load(f)
        success_count = 0
        for url, metadata in metadata_dict.items():
            notebook_name = metadata.get('notebook_name')
            if not notebook_name:
                print(f"Skipping entry with missing notebook_name in metadata: {url}")
                continue

            # Possible file locations
            possible_paths = [
                os.path.join(folder_path,competition_id, f"{notebook_name}.ipynb"),
            ]
            notebook_file = next((p for p in possible_paths if os.path.isfile(p)), None)

            if not notebook_file:
                print(f"Notebook file not found for {notebook_name}")
                continue

            # Add URL back to metadata
            metadata['url'] = url

            if self.import_notebook(notebook_file, competition_id, metadata):
                success_count += 1
                print(f"Imported notebook: {notebook_name}")
            else:
                print(f"Failed to import notebook: {notebook_name}")
        
        print(f"Successfully imported {success_count} of {len(metadata_dict)} notebooks")
        return success_count, len(metadata_dict)
    
    # Dataset methods
    def import_dataset(self, file_path, competition_id, dataset_type='train'):
        """Import a dataset (CSV file) into MongoDB and associate with a competition.
        
        Args:
            file_path (str): Path to the CSV file
            competition_id (str): ID of the competition to associate with
            dataset_type (str): Type of dataset - 'train', 'test', or 'validation'
        """
        try:
            # Check if competition exists
            if not self.get_competition(competition_id):
                print(f"Competition {competition_id} doesn't exist")
                return False
                
            # Check if file is a CSV
            if not file_path.endswith('.csv'):
                print(f"Skipping {file_path}: Not a CSV file")
                return False
            
            # Extract filename without extension as dataset_id
            dataset_id = os.path.splitext(os.path.basename(file_path))[0]
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            records = df.to_dict('records')
            
            # Store dataset metadata
            metadata = {
                'dataset_id': dataset_id,
                'competition_id': competition_id,
                'dataset_type': dataset_type,
                'original_filename': os.path.basename(file_path),
                'row_count': len(records),
                'column_names': list(df.columns),
                'import_date': datetime.datetime.now()
            }
            
            # Store the dataset in MongoDB
            self.datasets.update_one(
                {'dataset_id': dataset_id, 'competition_id': competition_id},
                {
                    '$set': {
                        'metadata': metadata,
                        'records': records
                    }
                },
                upsert=True
            )
            
            print(f"Imported {dataset_type} dataset: {dataset_id} with {len(records)} records for competition: {competition_id}")
            return True
            
        except Exception as e:
            print(f"Error importing dataset {file_path}: {e}")
            return False
    
    # User profile methods
    def create_user_profile(self, username, email, additional_info=None):
        """Create a new user profile with a generated UUID."""
        try:
            user_id = str(uuid.uuid4())
            profile = {
                'user_id': user_id,
                'username': username,
                'email': email,
                'created_at': datetime.datetime.now(),
                'last_login': datetime.datetime.now(),
                'current_competition_id': None  # User is not in any competition initially
            }
            
            # Add any additional info if provided
            if additional_info and isinstance(additional_info, dict):
                profile.update(additional_info)
            
            self.user_profiles.insert_one(profile)
            print(f"Created user profile for {username} with ID: {user_id}")
            return user_id
            
        except Exception as e:
            print(f"Error creating user profile: {e}")
            return None
    
    def get_user_profile(self, user_id):
        """Retrieve user profile by user_id."""
        return self.user_profiles.find_one({'user_id': user_id}, {'_id': 0})
    
    def update_user_profile(self, user_id, update_data):
        """Update a user profile with new data."""
        try:
            result = self.user_profiles.update_one(
                {'user_id': user_id},
                {'$set': update_data}
            )
            if result.modified_count > 0:
                print(f"Updated user profile for {user_id}")
                return True
            else:
                print(f"User profile {user_id} not found or no changes needed")
                return False
        except Exception as e:
            print(f"Error updating user profile: {e}")
            return False
    
    def set_user_active_competition(self, user_id, competition_id):
        """Set the active competition for a user."""
        try:
            # Verify both user and competition exist
            user = self.user_profiles.find_one({'user_id': user_id})
            if not user:
                print(f"User {user_id} not found")
                return False
                
            competition = self.competitions.find_one({'competition_id': competition_id})
            if not competition:
                print(f"Competition {competition_id} not found")
                return False
            
            # Update user's active competition
            result = self.user_profiles.update_one(
                {'user_id': user_id},
                {'$set': {
                    'current_competition_id': competition_id,
                    'competition_joined_at': datetime.datetime.now()
                }}
            )
            
            # Initialize or update the competition history for this user
            self.competition_history.update_one(
                {'user_id': user_id},
                {'$set': {
                    'user_id': user_id,
                    'competition_id': competition_id,
                    'start_time': datetime.datetime.now(),
                    'last_activity': datetime.datetime.now(),
                    'current_round': 1,
                    'conversation_history': [],
                    'submission_history': []
                }},
                upsert=True
            )
            
            print(f"Set active competition {competition_id} for user {user_id}")
            return True
            
        except Exception as e:
            print(f"Error setting active competition: {e}")
            return False
    
    def get_user_active_competition(self, user_id):
        """Get the active competition for a user."""
        user = self.user_profiles.find_one({'user_id': user_id}, {'current_competition_id': 1, '_id': 0})
        if user and user.get('current_competition_id'):
            return self.competitions.find_one(
                {'competition_id': user['current_competition_id']},
                {'_id': 0}
            )
        return None
    
    # Competition history methods
    def log_conversation(self, user_id, message, is_user=True):
        """Log a conversation message between user and chatbot for the active competition."""
        try:
            # Get user's active competition
            user = self.user_profiles.find_one({'user_id': user_id})
            if not user or not user.get('current_competition_id'):
                print(f"User {user_id} not found or has no active competition")
                return False
            
            # Create message entry
            message_entry = {
                'timestamp': datetime.datetime.now(),
                'is_user': is_user,
                'content': message
            }
            
            # Add message to conversation history
            result = self.competition_history.update_one(
                {'user_id': user_id},
                {
                    '$push': {'conversation_history': message_entry},
                    '$set': {'last_activity': datetime.datetime.now()}
                }
            )
            
            if result.modified_count > 0:
                print(f"Logged conversation for user {user_id}")
                return True
            else:
                print(f"Competition history for user {user_id} not found")
                return False
            
        except Exception as e:
            print(f"Error logging conversation: {e}")
            return False
    
    def advance_competition_round(self, user_id):
        """Advance to the next round in the competition for a user."""
        try:
            # Get current competition history
            history = self.competition_history.find_one({'user_id': user_id})
            if not history:
                print(f"Competition history for user {user_id} not found")
                return False
            
            # Advance to next round
            current_round = history.get('current_round', 1)
            next_round = current_round + 1
            
            result = self.competition_history.update_one(
                {'user_id': user_id},
                {
                    '$set': {
                        'current_round': next_round,
                        'round_start_time': datetime.datetime.now()
                    }
                }
            )
            
            if result.modified_count > 0:
                print(f"Advanced user {user_id} to round {next_round}")
                return True
            else:
                print(f"Failed to advance round for user {user_id}")
                return False
            
        except Exception as e:
            print(f"Error advancing competition round: {e}")
            return False
    
    def log_submission(self, user_id, notebook_id=None, score=None, submission_notes=None):
        """Log a competition submission for a user."""
        try:
            # Get user's active competition
            user = self.user_profiles.find_one({'user_id': user_id})
            if not user or not user.get('current_competition_id'):
                print(f"User {user_id} not found or has no active competition")
                return False
            
            competition_id = user['current_competition_id']
            
            # Get current competition history
            history = self.competition_history.find_one({'user_id': user_id})
            if not history:
                print(f"Competition history for user {user_id} not found")
                return False
            
            # Create submission entry
            submission = {
                'submission_id': str(uuid.uuid4()),
                'timestamp': datetime.datetime.now(),
                'round': history.get('current_round', 1),
                'notebook_id': notebook_id,
                'score': score,
                'notes': submission_notes
            }
            
            # Add submission to history
            result = self.competition_history.update_one(
                {'user_id': user_id},
                {
                    '$push': {'submission_history': submission},
                    '$set': {'last_activity': datetime.datetime.now()}
                }
            )
            
            if result.modified_count > 0:
                print(f"Logged submission for user {user_id} in competition {competition_id}")
                return submission['submission_id']
            else:
                print(f"Failed to log submission for user {user_id}")
                return None
            
        except Exception as e:
            print(f"Error logging submission: {e}")
            return None
    
    def get_user_competition_history(self, user_id):
        """Get the complete competition history for a user."""
        return self.competition_history.find_one({'user_id': user_id}, {'_id': 0})
    
    def get_user_conversation_history(self, user_id, limit=None):
        """Get conversation history for a user's active competition."""
        history = self.competition_history.find_one({'user_id': user_id}, {'conversation_history': 1, '_id': 0})
        if history and 'conversation_history' in history:
            conversations = history['conversation_history']
            if limit:
                return conversations[-limit:]
            return conversations
        return []
    
    def end_competition_session(self, user_id, final_score=None, notes=None):
        """End a user's competition session."""
        try:
            # Get user's active competition
            user = self.user_profiles.find_one({'user_id': user_id})
            if not user or not user.get('current_competition_id'):
                print(f"User {user_id} not found or has no active competition")
                return False
            
            competition_id = user['current_competition_id']
            
            # Update competition history
            end_time = datetime.datetime.now()
            
            result = self.competition_history.update_one(
                {'user_id': user_id},
                {
                    '$set': {
                        'end_time': end_time,
                        'final_score': final_score,
                        'completion_notes': notes,
                        'status': 'completed'
                    }
                }
            )
            
            # Clear the user's active competition
            self.user_profiles.update_one(
                {'user_id': user_id},
                {'$set': {'current_competition_id': None}}
            )
            
            if result.modified_count > 0:
                print(f"Ended competition session for user {user_id}")
                return True
            else:
                print(f"Failed to end competition session for user {user_id}")
                return False
            
        except Exception as e:
            print(f"Error ending competition session: {e}")
            return False
    
    def get_competition(self, competition_id):
        """Get competition details by ID."""
        return self.competitions.find_one({'competition_id': competition_id}, {'_id': 0})
    
    def list_competitions(self):
        """List all competitions."""
        return list(self.competitions.find({}, {'_id': 0}))
    
    def get_competition_datasets(self, competition_id):
        """Get all datasets associated with a competition."""
        return list(self.datasets.find({'competition_id': competition_id}, {'_id': 0}))
    
    def get_competition_notebooks(self, competition_id):
        """Get all notebooks associated with a competition."""
        return list(self.notebooks.find({'competition_id': competition_id}, {'_id': 0, 'content': 0}))
    def get_notebook(self, notebook_id, competition_id):
        """Retrieve a notebook by its ID and competition ID."""
        return self.notebooks.find_one({
            'notebook_id': notebook_id,
            'competition_id': competition_id
        }, {'_id': 0})
    
    def export_notebook(self, notebook_id, competition_id, output_path):
        """Export a notebook to a .ipynb file."""
        notebook = self.get_notebook(notebook_id, competition_id)
        if notebook and 'content' in notebook:
            try:
                with open(output_path, 'w', encoding='utf-8') as f:
                    json.dump(notebook['content'], f)
                print(f"Notebook exported to {output_path}")
                return True
            except Exception as e:
                print(f"Error exporting notebook: {e}")
                return False
        else:
            print(f"Notebook {notebook_id} for competition {competition_id} not found or has no content")
            return False
        
        
    def get_dataset(self, dataset_id, competition_id):
        """Retrieve a dataset by its ID and competition ID."""
        return self.datasets.find_one({
            'metadata.dataset_id': dataset_id,
            'metadata.competition_id': competition_id
        }, {'_id': 0})
    
    def get_competition_dataset_by_type(self, competition_id, dataset_type='train'):
        """Get a specific type of dataset for a competition."""
        return self.datasets.find_one({
            'metadata.competition_id': competition_id,
            'metadata.dataset_type': dataset_type
        }, {'_id': 0})
    
    def get_dataset_as_dataframe(self, dataset_id, competition_id):
        """Retrieve a dataset as a pandas DataFrame."""
        dataset = self.get_dataset(dataset_id, competition_id)
        if dataset and 'records' in dataset:
            return pd.DataFrame(dataset['records'])
        return None
    
    def export_dataset(self, dataset_id, competition_id, output_path):
        """Export a dataset to a CSV file."""
        df = self.get_dataset_as_dataframe(dataset_id, competition_id)
        if df is not None:
            try:
                df.to_csv(output_path, index=False)
                print(f"Dataset exported to {output_path}")
                return True
            except Exception as e:
                print(f"Error exporting dataset: {e}")
                return False
        else:
            print(f"Dataset {dataset_id} for competition {competition_id} not found")
            return False

In [38]:
dm = DataManager()

# Create Compeition Collections

In [13]:
with open('/Users/zhongming/Local Docs/Github/Kaggle_RAG_dataset/data/test/competitions_metadata.json','r') as file:
    competitions_data = json.load(file)

# Import each competition
success_count = 0
for competition_id, competition_info in competitions_data.items():
    # Create a competition record with required fields
    competition_record = {
        'competition_id': competition_id,
        'title': competition_id.title(),  # Use ID as title if not provided
        'description': competition_info.get('Description', ''),
        'evaluation': competition_info.get('Evaluation', ''),
        'competition_host': competition_info.get('Competition Host', []),
        'price_award': competition_info.get('Prizes & Awards', []),
        'entrants': competition_info.get('Entrants', 0),
        'participants': competition_info.get('Participants', 0),
        'teams': competition_info.get('Teams', 0),
        'submissions': competition_info.get('Submissions', 0),
        'tags': competition_info.get('Tags', []),
        'competition_url': competition_info.get('competition_url', '')
    }
    
    # Add data description if available
    if 'data' in competition_info:
        data_info = competition_info['data']
        competition_record['data_description'] = data_info.get('Description', '')
        competition_record['data_files_num'] = data_info.get('Files', '')
        competition_record['data_size'] = data_info.get('Size', '')
        competition_record['data_type'] = data_info.get('Type', '')
    
    # Create competition in MongoDB
    if dm.create_competition(competition_record):
        success_count += 1
        print(f"Successfully imported competition: {competition_id}")
    else:
        print(f"Failed to import competition: {competition_id}")

print(f"\nSummary: Successfully imported {success_count} of {len(competitions_data)} competitions")


Successfully imported competition: titanic

Summary: Successfully imported 1 of 1 competitions


In [14]:
dm.get_competition('titanic')

{'competition_id': 'titanic',
 'competition_host': ['Kaggle'],
 'competition_url': 'https://www.kaggle.com/competitions/titanic',
 'data_description': 'Dataset Description\nOverview\nThe data has been split into two groups:\ntraining set (train.csv)\ntest set (test.csv)\nThe training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.\nThe test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.\nWe also include gender_submission.csv, a set of predictions that assume all and only female pas

# Create dataset collection for train and test set of competition

In [17]:
base_dir = '/Users/zhongming/Local Docs/Github/Kaggle_RAG_dataset/data/dataset' 
# Get all competition directories
competition_dirs = [d for d in os.listdir(base_dir) 
                    if os.path.isdir(os.path.join(base_dir, d))]


In [ ]:
base_dir = '/Users/zhongming/Local Docs/Github/Kaggle_RAG_dataset/data/dataset' 
# Get all competition directories
competition_dirs = [d for d in os.listdir(base_dir) 
                    if os.path.isdir(os.path.join(base_dir, d))]

total_datasets = 0
total_imported = 0

# Process each competition directory
for competition_id in competition_dirs:
    competition_dir = os.path.join(base_dir, competition_id)
    
    # Check if competition exists in the database
    competition = dm.get_competition(competition_id)
    if not competition:
        print(f"Competition {competition_id} does not exist in the database, skipping...")
        continue
    
    # Find all CSV files in the competition directory
    csv_pattern = os.path.join(competition_dir, "*.csv")
    csv_files = glob.glob(csv_pattern)
    
    if not csv_files:
        print(f"No CSV files found for competition {competition_id}")
        continue
    
    print(f"\nProcessing competition: {competition_id}")
    print(f"Found {len(csv_files)} CSV files")
    
    # Import each CSV file
    imported_count = 0
    for csv_file in csv_files:
        file_name = os.path.basename(csv_file)
        base_name = os.path.splitext(file_name)[0]
        
        # Determine dataset type based on filename
        dataset_type = 'unknown'
        if 'train' in base_name.lower():
            dataset_type = 'train'
        elif 'test' in base_name.lower():
            dataset_type = 'test'
        elif 'valid' in base_name.lower() or 'val' in base_name.lower():
            dataset_type = 'validation'
        
        print(f"Importing {file_name} as {dataset_type} dataset...")
        
        # Import dataset
        if dm.import_dataset(csv_file, competition_id, dataset_type):
            imported_count += 1
            print(f"Successfully imported {file_name}")
        else:
            print(f"Failed to import {file_name}")
    
    print(f"Imported {imported_count} of {len(csv_files)} datasets for competition {competition_id}")
    
    total_datasets += len(csv_files)
    total_imported += imported_count


Processing competition: titanic
Found 3 CSV files
Importing test.csv as test dataset...
Imported test dataset: test with 418 records for competition: titanic
Successfully imported test.csv
Importing train.csv as train dataset...
Imported train dataset: train with 891 records for competition: titanic
Successfully imported train.csv
Importing gender_submission.csv as unknown dataset...
Imported unknown dataset: gender_submission with 418 records for competition: titanic
Successfully imported gender_submission.csv
Imported 3 of 3 datasets for competition titanic


In [22]:
pd.DataFrame(dm.get_competition_dataset_by_type('titanic','train')['records'])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Create notebook collections

In [41]:
dm = DataManager()

In [42]:
base_dir = '/Users/zhongming/Local Docs/Github/Kaggle_RAG_dataset/data/notebooks'
# Get all competition directories
competition_dirs = [d for d in os.listdir(base_dir) 
                    if os.path.isdir(os.path.join(base_dir, d))]

In [43]:
total_notebooks = 0
total_imported = 0

# Process each competition directory
for competition_id in competition_dirs:
    # Import each notebook file
    success_count, total_count = dm.import_notebook_folder(base_dir, competition_id)
    
    total_notebooks += total_count
    total_imported += success_count

# Print summary
print(f"\nSummary: Successfully imported {total_imported} of {total_notebooks} notebooks")

Imported notebook: vladimirsydor_add-leak
Imported notebook: yunishi0716_best-weight-searching3
Imported notebook: aleksthegreat_public-blend
Imported notebook: yamsam_ashrae-leak-validation-and-more
Imported notebook: wuliaokaola_ashrae-maybe-this-can-make-public-lb-some-useful
Imported notebook: vladimirsydor_bland-lgbm-on-leaks
Imported notebook: vladimirsydor_bland-by-leak
Imported notebook: rohanrao_ashrae-divide-and-conquer
Imported notebook: teeyee314_best-single-half-half-lgbm-1-07
Imported notebook: vladimirsydor_bland-lgbm-folds
Imported notebook: mimoudata_ashrae-2-lightgbm-without-leak-data
Imported notebook: aitude_ashrae-kfold-lightgbm-without-leak-1-08
Imported notebook: purist1024_ashrae-simple-data-cleanup-lb-1-08-no-leaks
Imported notebook: ragnar123_another-1-08-lb-no-leak
Imported notebook: mimoudata_ashrae-lightgbm-without-leak
Imported notebook: yunishi0716_k-folds-model
Imported notebook: hmendonca_4-ashrae-blended
Imported notebook: grapestone5321_ashrae-stackin

# Create user profile collection

In [44]:
dm.create_user_profile('test_user', 'test@test.com')

Created user profile for test_user with ID: 8c13b49d-8a41-4dea-9bd4-32f96466a08e


'8c13b49d-8a41-4dea-9bd4-32f96466a08e'

# Create competition_history collection

#